<a href="https://colab.research.google.com/github/Asad-Khan9/Capstone-Research/blob/main/notebook1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate torch scikit-learn nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load GPT-2 model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load a smaller model (e.g., distilgpt2)
small_model_name = "distilgpt2"
small_model = AutoModelForCausalLM.from_pretrained(small_model_name)
small_tokenizer = AutoTokenizer.from_pretrained(small_model_name)

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

# Load a dataset (e.g., wikitext for language modeling)
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")

# Load model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def calculate_perplexity(model, tokenizer, dataset, device, max_length=512):
    model.eval()
    total_loss = 0
    total_length = 0

    with torch.no_grad():
        for text in dataset["text"]:
            # Skip empty texts
            if not text.strip():
                continue

            # Tokenize text
            inputs = tokenizer(
                text,
                return_tensors="pt",
                max_length=max_length,
                truncation=True,
                return_attention_mask=True
            )

            # Skip if input_ids is empty
            if inputs["input_ids"].numel() == 0:
                continue

            # Move tensors to device
            inputs = {k: v.to(device) for k, v in inputs.items()}

            # Compute loss
            outputs = model(**inputs, labels=inputs["input_ids"])
            loss = outputs.loss

            # Accumulate loss and length
            seq_length = inputs["input_ids"].size(1)
            total_loss += loss.item() * seq_length
            total_length += seq_length

    # Handle edge case where all texts are empty
    if total_length == 0:
        return float("inf")  # Return infinity if no valid texts

    perplexity = torch.exp(torch.tensor(total_loss / total_length)).item()
    return perplexity

# Calculate perplexity
perplexity = calculate_perplexity(model, tokenizer, dataset, device)
print("GPT-2 Perplexity:", perplexity)

GPT-2 Perplexity: 50.07951736450195


In [ ]:
import evaluate
from tqdm import tqdm

def evaluate_generation_metrics(model, tokenizer, dataset, device, max_length=50, num_samples=100):
    model.eval()
    bleu = evaluate.load("bleu")
    rouge = evaluate.load("rouge")

    predictions = []
    references = []

    with torch.no_grad():
        # Process a subset of the dataset for efficiency
        for text in tqdm(dataset["text"][:num_samples], desc="Evaluating generation"):
            if not text.strip():
                continue

            # Split text into prompt (first 50 tokens) and reference (remaining tokens)
            tokens = tokenizer.encode(text, return_tensors="pt", truncation=True, max_length=200).to(device)
            if tokens.size(1) < 100:
                continue

            prompt_tokens = tokens[:, :50]
            reference_tokens = tokens[:, 50:100]

            try:
                # Generate continuation
                generated = model.generate(
                    prompt_tokens,
                    max_length=100,
                    num_return_sequences=1,
                    pad_token_id=tokenizer.eos_token_id
                )

                # Decode texts
                prompt_text = tokenizer.decode(prompt_tokens[0], skip_special_tokens=True)
                generated_text = tokenizer.decode(generated[0][50:], skip_special_tokens=True)  # Remove prompt
                reference_text = tokenizer.decode(reference_tokens[0], skip_special_tokens=True)

                predictions.append(generated_text)
                references.append([reference_text])  # BLEU expects list of lists
            except Exception as e:
                print(f"Skipping due to error: {e}")
                continue

    # Calculate metrics
    bleu_scores = bleu.compute(predictions=predictions, references=references)
    rouge_scores = rouge.compute(predictions=predictions, references=references)

    return bleu_scores, rouge_scores
# First load the dataset properly
from datasets import load_dataset

# Load the test split explicitly
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")  # No ["test"] needed here

# Then modify the evaluation calls to:
# Evaluate GPT-2
gpt2_bleu, gpt2_rouge = evaluate_generation_metrics(model, tokenizer, dataset, device)  # Pass dataset directly
print("GPT-2 BLEU:", gpt2_bleu)
print("GPT-2 ROUGE:", gpt2_rouge)

# Evaluate small model
small_bleu, small_rouge = evaluate_generation_metrics(small_model, small_tokenizer, dataset, device)  # Pass dataset directly
print("\nSmall Model BLEU:", small_bleu)
print("Small Model ROUGE:", small_rouge)

ImportError: To be able to use evaluate-metric/rouge, you need to install the following dependencies['rouge_score'] using 'pip install rouge_score' for instance'

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# Example predictions and labels
gpt2_predictions = [1, 0, 1, 1]
small_model_predictions = [1, 1, 0, 1]
labels = [1, 0, 1, 0]

# Compute accuracy and F1 score
gpt2_accuracy = accuracy_score(labels, gpt2_predictions)
small_accuracy = accuracy_score(labels, small_model_predictions)

gpt2_f1 = f1_score(labels, gpt2_predictions)
small_f1 = f1_score(labels, small_model_predictions)

print("GPT-2 Accuracy:", gpt2_accuracy)
print("Small Model Accuracy:", small_accuracy)
print("GPT-2 F1 Score:", gpt2_f1)
print("Small Model F1 Score:", small_f1)